In [387]:
import pandas as pd
import numpy as np
#from scipy.stats import beta # Beta probability function
from scipy.special import beta # Beta raw function
import os
from scipy.optimize import minimize

In [388]:
os.chdir('C:/Users/renan/Desktop/inequality-segregation-supplementary-files/')

In [389]:
census_2010 = pd.read_csv('data/std_2010_fullcount.csv', encoding = "ISO-8859-1", sep = ",")
df = census_2010.loc[census_2010.county == "Riverside County"][['trtid10','tract','pop10','nhblk10']]
df.sort_values('pop10').head()

,trtid10,tract,pop10,nhblk10
6073,6065980004,Census Tract 9800.04,9,0
12813,6065940100,Census Tract 9401,166,3
6027,6065043814,Census Tract 438.14,905,26
12595,6065044521,Census Tract 445.21,1196,54
12341,6065030101,Census Tract 301.01,1226,72


In [390]:
# M_set = {m_1, ..., m_r} 
M_set = [1, 2, 3, 4, 5, 6, 6242]

# Assuming c Beta distributions
c = 2

In [391]:
n =  0 # 3 #677
m =  9 # 166 #M_set[6]

# Count the number of units of size m with n minority individuals
A_n_m = len(np.where((df['pop10'] == m) & (df['nhblk10'] == n))[0].tolist())

def internal_likelihood_evaluator(lambda_j, alpha_j, beta_j):
    result = lambda_j * beta(alpha_j + n, beta_j + m - n) / beta(alpha_j, beta_j)
    return result

lambdas = [0.1, 0.9]
alphas  = [0.5, 0.45]
betas   = [0.7, 0.1]

inner_sum = np.array(list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))).sum()

A_n_m * np.log(inner_sum)

-2.520981363054838

In [392]:
list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))

[0.023005810535624515, 0.05737487486002151]

In [393]:
#for m in M_set:

def evaluate_likelihood_for_n(n):
    A_n_m = len(np.where((df['pop10'] == m) & (df['nhblk10'] == n))[0].tolist())

    def internal_likelihood_evaluator(lambda_j, alpha_j, beta_j):
        result = lambda_j * beta(alpha_j + n, beta_j + m - n) / beta(alpha_j, beta_j)
        return result

    inner_sum = np.array(list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))).sum()

    return A_n_m * np.log(inner_sum)

In [394]:
m1 = 5

range(m1)

list(map(lambda x: evaluate_likelihood_for_n(x), range(m1)))

[-2.520981363054838, -0.0, -0.0, -0.0, -0.0]

In [395]:
lambdas = [0.1, 0.9]
alphas  = [0.5, 0.45]
betas   = [0.7, 0.1]

def evaluate_likelihood_for_m(m):
    
    def evaluate_likelihood_for_n(n):

        def internal_likelihood_evaluator(lambda_j, alpha_j, beta_j):
            result = lambda_j * beta(alpha_j + n, beta_j + m - n) / beta(alpha_j, beta_j)
            return result

        A_n_m = len(np.where((df['pop10'] == m) & (df['nhblk10'] == n))[0].tolist())
        #inner_sum = np.array(list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))).sum()
        inner_sum = np.nansum(np.array(list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))))
        
        #if np.isnan(np.log(inner_sum)):
        #    return 0
        #else:
        #    return A_n_m * np.log(inner_sum) # Add some noise + 0.001 to avoid 0 evaluation in the log
        try:
            return A_n_m * np.log(inner_sum)
        except:
            return 0
    
    aux = range(m)
    #outer_sum = np.array(list(map(lambda x: evaluate_likelihood_for_n(x), aux))).sum()
    outer_sum = np.nansum(np.array(list(map(lambda x: evaluate_likelihood_for_n(x), aux))))
    return outer_sum

In [396]:
evaluate_likelihood_for_m(9)

-2.520981363054838

In [397]:
evaluate_likelihood_for_m(166)

-18.663144501908384

In [398]:
lambdas = [0.5]
alphas = [0.5]
betas = [0.65]

In [399]:
evaluate_likelihood_for_m(9)

-2.2085369442748366

In [400]:
evaluate_likelihood_for_m(166)

-18.328023884738688

In [236]:
np.isnan(na)

True

In [401]:
M_set = [9, 166]

np.array(list(map(lambda x: evaluate_likelihood_for_m(x), M_set))).sum()

-20.536560829013524

In [402]:
def evaluate_likelihood_for_M_set(m_set):
    # m_set is a list of m's
    return np.array(list(map(lambda x: evaluate_likelihood_for_m(x), M_set))).sum()

In [253]:
evaluate_likelihood_for_M_set(M_set)

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


-21.18412586496322

In [418]:
M_set = [9, 9]#166]#, 905]

def objective_function(params):
    #M_set = M_set
    
    def evaluate_likelihood_for_m(m):
    
        def evaluate_likelihood_for_n(n):

            def internal_likelihood_evaluator(lambda_j, alpha_j, beta_j):
                result = lambda_j * beta(alpha_j + n, beta_j + m - n) / beta(alpha_j, beta_j)
                return result

            #alphas  = alpha_j
            #betas   = beta_j
            #lambdas = lambda_j     
            
            alphas, betas, lambdas = params
            alphas  = [alphas]
            betas   = [betas]
            lambdas = [lambdas]

            A_n_m = len(np.where((df['pop10'] == m) & (df['nhblk10'] == n))[0].tolist())
            inner_sum = np.nansum(np.array(list(map(lambda x, y, z: internal_likelihood_evaluator(x, y, z), lambdas, alphas, betas))))

            try:
                return A_n_m * np.log(inner_sum)
            except:
                return 0
    
        aux = range(m)
        outer_sum = np.nansum(np.array(list(map(lambda x: evaluate_likelihood_for_n(x), aux))))
        return outer_sum
    
    # Invert the value in order to use the minimize function
    return np.array(list(map(lambda x: evaluate_likelihood_for_m(x), M_set))).sum() * (-1)

In [419]:
lambda_j = 0.5
alpha_j = 0.5
beta_j = 0.75
result = lambda_j * beta(alpha_j + n, beta_j + m - n) / beta(alpha_j, beta_j)

A_n_m = len(np.where((df['pop10'] == m) & (df['nhblk10'] == n))[0].tolist())
A_n_m * np.log(inner_sum)

aux = range(m)
outer_sum = np.nansum(np.array(list(map(lambda x: evaluate_likelihood_for_n(x), aux))))
outer_sum

-2.2085369442748366

In [420]:
np.array(list(map(lambda x: evaluate_likelihood_for_m(x), M_set))).sum() * -1

4.417073888549673

In [423]:
initial_guesses = np.array([0.65, 0.5, 0.5])
res = minimize(objective_function, 
               initial_guesses, 
               method='nelder-mead',
               #method='Powell',
               options = {'xtol': 0.1})
               #options = {'xtol': 1e-8})

C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in log
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\optimize.py:563: RuntimeWarning: invalid value encountered in subtract
  numpy.max(numpy.abs(fsim[0] - fsim[1:])) <= fatol):
C:\Users\renan\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [424]:
res#.final_simplex#[0][1] # Point where min

 final_simplex: (array([[-0.83123202,  0.83123202,  1.14972094],
       [-0.83123202,  0.83123202,  1.14972094],
       [-0.83123202,  0.83123202,  1.14972094],
       [-0.83123202,  0.83123202,  1.14972094]]), array([-inf, -inf, -inf, -inf]))
           fun: -inf
       message: 'Maximum number of function evaluations has been exceeded.'
          nfev: 603
           nit: 232
        status: 1
       success: False
             x: array([-0.83123202,  0.83123202,  1.14972094])

In [410]:
?minimize

In [302]:
lambdas = [0.1]#[0.1, 0.9]
alphas  = [0.5]#[0.5, 0.45]
betas   = [0.7]#[0.7, 0.1]
objective_function(alphas, betas, lambdas)

TypeError: objective_function() takes 1 positional argument but 3 were given

In [275]:
alphas, betas, lambdas = np.array([[0.5], [0.5], [0.5]])

In [280]:
objective_function([0.5, 0.5, 0.5])

TypeError: 'float' object is not iterable

In [277]:
betas

0.5

In [278]:
lambdas

0.5

In [324]:
beta(1,1)

1.0

In [327]:
beta(-0.5,14)

-12.904806261177237